In [5]:
import io
import numpy as np
import soundfile as sf
from kokoro import KPipeline

# Initialize Kokoro pipeline
pipeline = KPipeline(lang_code='a', repo_id='hexgrad/Kokoro-82M')

def generate_audio_from_text(text: str) -> bytes:
    if not text.strip():
        raise ValueError("Text input is empty.")
    
    generator = pipeline(text, voice='af_heart')
    all_audio = [audio for _, (_, _, audio) in enumerate(generator)]

    if not all_audio:
        raise ValueError("No audio generated")

    combined_audio = np.concatenate(all_audio)
    buffer = io.BytesIO()
    sf.write(buffer, combined_audio, 24000, format='WAV', subtype='PCM_16')
    buffer.seek(0)
    return buffer.read()


In [3]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

# Initialize Gemini client
api_key = os.environ.get("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY environment variable is not set")

gemini_client = genai.Client(api_key=api_key)

def generate_gemini_response(prompt: str, max_length: int = 500) -> str:
    if not prompt.strip():
        raise ValueError("Prompt input is empty.")

    model = "gemini-1.5-flash"
    contents = [
        types.Content(role="user", parts=[types.Part.from_text(text=prompt)]),
    ]
    config = types.GenerateContentConfig(response_mime_type="text/plain")

    response_text = ""
    for chunk in gemini_client.models.generate_content_stream(
        model=model, contents=contents, config=config
    ):
        chunk_text = chunk.text
        if len(response_text) + len(chunk_text) > max_length:
            response_text += chunk_text[:max_length - len(response_text)]
            break
        response_text += chunk_text
        if len(response_text) >= max_length:
            break

    return response_text


In [6]:
from IPython.display import Audio

# Generate audio bytes
audio_bytes = generate_audio_from_text("Hello students, welcome to AI learning.")

# Play in notebook
Audio(audio_bytes, rate=24000)


In [4]:
response = generate_gemini_response("What is the importance of learning Python?")
print(response)

Learning Python offers numerous advantages, making it a valuable skill in today's world.  Its importance stems from several key factors:

**1. Versatility and Wide Applicability:**

* **Web Development:** Python's frameworks like Django and Flask are powerful tools for building robust and scalable web applications.
* **Data Science and Machine Learning:**  Python is the dominant language in this field, with libraries like NumPy, Pandas, Scikit-learn, and TensorFlow providing extensive capabiliti
